# **Feature Set 3 - Dynamic Data preprocessing**

We will now carry out preprocessing of the raw data extracted based on the features identified in our second feature set plus our first feature set.

The logic and steps used to preprocess the data for the additional features will be identical to that used for feature set 1 and 2 dynamic features.

When the data is split into the train and test set, we will include the same patients in the train and test set as we have used for all analysis with feature set 1 for consistency.

**Decision - use the exact same patient-wise train and test split**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### **Step 1: Load the time-series data**

In [ ]:
full_data = '/content/drive/MyDrive/MSc_Final_Project/02_data_analysis/mimic/data_analysis/datasets/06_extracted_data_analysis/03_feature_set_3_analysis/patient_f3_dynamic_data_v2.parquet'

patient_data = pd.read_parquet(full_data)

patient_data.head()

,subject_id,itemid,valuenum,time_from_window_start_mins,feature_label
0,10001884,223835,40.0,200.0,Inspired O2 Fraction
1,10001884,224685,284.0,200.0,Tidal Volume (observed)
2,10001884,224686,284.0,200.0,Tidal Volume (spontaneous)
3,10001884,224687,6.1,200.0,Minute Volume
4,10001884,224695,17.0,200.0,Peak Insp. Pressure


In [ ]:
# Drop the itemid column
patient_data = patient_data.drop(columns=['itemid'])

patient_data.head()

,subject_id,valuenum,time_from_window_start_mins,feature_label
0,10001884,40.0,200.0,Inspired O2 Fraction
1,10001884,284.0,200.0,Tidal Volume (observed)
2,10001884,284.0,200.0,Tidal Volume (spontaneous)
3,10001884,6.1,200.0,Minute Volume
4,10001884,17.0,200.0,Peak Insp. Pressure


In [ ]:
# Identify all unique fature_labels
feature_labels = patient_data['feature_label'].unique()

print(len(feature_labels))

print(feature_labels)

43
['Inspired O2 Fraction' 'Tidal Volume (observed)'
 'Tidal Volume (spontaneous)' 'Minute Volume' 'Peak Insp. Pressure'
 'Mean Airway Pressure' 'EtCO2' 'Heart Rate' 'Respiratory Rate'
 'GCS - Eye Opening' 'GCS - Motor Response' 'O2 saturation pulseoxymetry'
 'Richmond-RAS Scale' 'Ventilator Mode' 'Arterial Blood Pressure systolic'
 'Arterial Blood Pressure diastolic' 'Arterial Blood Pressure mean'
 'Temperature Fahrenheit' 'Hematocrit (serum)' 'Sodium (serum)'
 'Potassium (serum)' 'Arterial O2 pressure' 'Arterial CO2 Pressure'
 'PH (Arterial)' 'Arterial Base Excess' 'Arterial O2 Saturation'
 'Ionized Calcium' 'Lactic Acid' 'Hemoglobin' 'WBC' 'Creatinine (serum)'
 'Glucose (serum)' 'Platelet Count' 'CO2 production' 'Compliance'
 'Plateau Pressure' 'Mixed Venous O2% Sat' 'PH (Venous)'
 'Venous CO2 Pressure' 'Venous O2 Pressure' 'Cardiac Output (CCO)'
 'Total Bilirubin' 'Negative Insp. Force']


We have all relevent features from feature set 1, 2 and 3.

In [ ]:
# Count the number of unique subject_ids
unique_subject_ids = patient_data['subject_id'].nunique()

print(unique_subject_ids)

4701


We also have the same number of patients in the global dataset as with feature set 1 so we can do the same 80/20 train/test split.

In [ ]:
patient_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 254916 entries, 0 to 254915
Data columns (total 4 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   subject_id                   254916 non-null  int64  
 1   valuenum                     254117 non-null  float64
 2   time_from_window_start_mins  254916 non-null  float64
 3   feature_label                254916 non-null  object 
dtypes: float64(2), int64(1), object(1)
memory usage: 7.8+ MB


### **Step 2: Remove low observed features**

As before, we will remove any features that have an observation frequency per patient below 0.5 in the 6 hours.

In [ ]:
# Calculate the average sampling frequency
per_patient_sampling_frequency = patient_data.groupby(['subject_id', 'feature_label']).size().reset_index(name='count')

per_patient_sampling_frequency_pivot = per_patient_sampling_frequency.pivot(index='subject_id', columns='feature_label', values='count').fillna(0)

# Calculate the average sampling frequency
average_sampling_frequency = per_patient_sampling_frequency_pivot.mean().sort_values(ascending=False)

# Create columns for the table
average_sampling_frequency_df = pd.DataFrame({'Feature': average_sampling_frequency.index, 'Average Sampling Frequency': average_sampling_frequency.values})

# Display the table
average_sampling_frequency_df

,Feature,Average Sampling Frequency
0,Heart Rate,6.647522
1,Respiratory Rate,6.614763
2,O2 saturation pulseoxymetry,6.610508
3,Arterial Blood Pressure mean,3.761115
4,Arterial Blood Pressure diastolic,3.754308
5,Arterial Blood Pressure systolic,3.753669
6,Inspired O2 Fraction,2.104446
7,GCS - Eye Opening,1.633057
8,GCS - Motor Response,1.628377
9,Tidal Volume (observed),1.600298


There are a lot of features with a significantly low observation frequency.

This reflects the poor sampling in the MIMIC data.

# **AMENDMENT**

**We have amended the threshold for feature set 3 to 0.15 in 6 hours in line with Zeng et al so that we can include more features in feature set 3 model training. **

In [ ]:
# Define the threshold
threshold = 0.15

In [ ]:
# Identify all the features below the threshold
low_observed_features = average_sampling_frequency_df[average_sampling_frequency_df['Average Sampling Frequency'] < threshold]['Feature'].tolist()
low_observed_features

['Compliance',
 'CO2 production',
 'Total Bilirubin',
 'PH (Venous)',
 'Venous O2 Pressure',
 'Mixed Venous O2% Sat',
 'Venous CO2 Pressure',
 'Negative Insp. Force']

In [ ]:
len(low_observed_features)

8

There are 22 features that have observation frequencies below the threshold. (Now 8 with the lower threshold)

We will now remove all data points associated with these features to minimise the amount of data synthesis that will be required later.

In [ ]:
patient_data.shape[0]

254916

In [ ]:
# From the patient data remove all rows that correspond to the low frequency features
patient_data_filtered_df = patient_data[~patient_data['feature_label'].isin(low_observed_features)]
patient_data_filtered_df.head()

,subject_id,valuenum,time_from_window_start_mins,feature_label
0,10001884,40.0,200.0,Inspired O2 Fraction
1,10001884,284.0,200.0,Tidal Volume (observed)
2,10001884,284.0,200.0,Tidal Volume (spontaneous)
3,10001884,6.1,200.0,Minute Volume
4,10001884,17.0,200.0,Peak Insp. Pressure


In [ ]:
patient_data_filtered_df.shape[0]

print("Number of rows removed:", patient_data.shape[0] - patient_data_filtered_df.shape[0])

Number of rows removed: 2320


In [ ]:
# Check that there are no entries of these features remaining
print(patient_data_filtered_df[patient_data_filtered_df['feature_label'].isin(low_observed_features)])

Empty DataFrame
Columns: [subject_id, valuenum, time_from_window_start_mins, feature_label]
Index: []


In [ ]:
# Number of features left
patient_data_filtered_df['feature_label'].nunique()

35

In [ ]:
# Save progress for now in drive
patient_data_filtered_df.to_parquet('/content/drive/MyDrive/MSc_Final_Project/02_data_analysis/mimic/data_analysis/datasets/07_data_preprocessing/03_feature_set_3/pre_processing/01_patient_data_f3_filtered')

### **Step 3: Split the data into train and test sets**

We will split the data using the same patients in the train and test sets used for feature set 1 for consistency.

In [ ]:
# First we will attach extubation failure labels to our patients
annotations_path = '/content/drive/MyDrive/MSc_Final_Project/02_data_analysis/mimic/data_analysis/datasets/03_annotated_set/annotation_v03.parquet'
annotations_df = pd.read_parquet(annotations_path)

patient_data_filtered_df.loc[:, 'extubation_failure'] = patient_data_filtered_df['subject_id'].map(annotations_df.set_index('subject_id')['extubation_failure'])
patient_data_filtered_df.head()

<ipython-input-75-0d768f3c7c6c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  patient_data_filtered_df.loc[:, 'extubation_failure'] = patient_data_filtered_df['subject_id'].map(annotations_df.set_index('subject_id')['extubation_failure'])


,subject_id,valuenum,time_from_window_start_mins,feature_label,extubation_failure
0,10001884,40.0,200.0,Inspired O2 Fraction,1
1,10001884,284.0,200.0,Tidal Volume (observed),1
2,10001884,284.0,200.0,Tidal Volume (spontaneous),1
3,10001884,6.1,200.0,Minute Volume,1
4,10001884,17.0,200.0,Peak Insp. Pressure,1


In [ ]:
# Load the train and test sets from feature set 1 preprocessing
train_path = '/content/drive/MyDrive/MSc_Final_Project/02_data_analysis/mimic/data_analysis/datasets/07_data_preprocessing/01_feature_set_1/01_preprocessing_v2/03_train_data_standard_preprocess_done.parquet'
test_path = '/content/drive/MyDrive/MSc_Final_Project/02_data_analysis/mimic/data_analysis/datasets/07_data_preprocessing/01_feature_set_1/01_preprocessing_v2/03_test_data_standard_preprocess_done.parquet'

train_f1 = pd.read_parquet(train_path)
test_df = pd.read_parquet(test_path)

train_f1.head()

,subject_id,itemid,valuenum,time_to_extubation_mins,time_from_window_start,label,extubation_failure
0,10001884,223835,40.0,160.0,200.0,Inspired O2 Fraction,1
1,10001884,224685,NaN,160.0,200.0,Tidal Volume (observed),1
2,10001884,224686,NaN,160.0,200.0,Tidal Volume (spontaneous),1
3,10001884,224687,6.1,160.0,200.0,Minute Volume,1
4,10001884,224695,17.0,160.0,200.0,Peak Insp. Pressure,1


In [ ]:
# Idetnify the patients in the train and test set
train_patients = train_f1['subject_id'].unique()
test_patients = test_df['subject_id'].unique()

print(len(train_patients))
print(len(test_patients))

3760
941


In [ ]:
# Split the patient data frame into train and test data by the train and test patients extracted
train_data = patient_data_filtered_df[patient_data_filtered_df['subject_id'].isin(train_patients)]
test_data = patient_data_filtered_df[patient_data_filtered_df['subject_id'].isin(test_patients)]

print(train_data.shape[0])
print(test_data.shape[0])

201718
50878


In [ ]:
print(train_data['subject_id'].nunique())
print(test_data['subject_id'].nunique())

3760
941


In [ ]:
train_data.head()

,subject_id,valuenum,time_from_window_start_mins,feature_label,extubation_failure
0,10001884,40.0,200.0,Inspired O2 Fraction,1
1,10001884,284.0,200.0,Tidal Volume (observed),1
2,10001884,284.0,200.0,Tidal Volume (spontaneous),1
3,10001884,6.1,200.0,Minute Volume,1
4,10001884,17.0,200.0,Peak Insp. Pressure,1


We have now split our data into the train and test set

In [ ]:
# Save progress so far
train_data.to_parquet('/content/drive/MyDrive/MSc_Final_Project/02_data_analysis/mimic/data_analysis/datasets/07_data_preprocessing/03_feature_set_3/pre_processing/02_train_data_f2.parquet')
test_data.to_parquet('/content/drive/MyDrive/MSc_Final_Project/02_data_analysis/mimic/data_analysis/datasets/07_data_preprocessing/03_feature_set_3/pre_processing/02_test_data_f2.parquet')


Let's see how much synthetic data will be required for our train and test split.

**The split is based on a maximal resampling rate of every 30 mins - which will not be the case for the medium and low frequency subsets but covers the max case scenario**

In [ ]:
def calculate_synthetic_data_proportion(df, patient_id_col, feature_col, target_frequency=13):
    synthetic_data_proportion = {}
    for patient_id in df[patient_id_col].unique():
        patient_data = df[df[patient_id_col] == patient_id]
        total_entries = len(patient_data)
        synthetic_entries = 0
        for feature in df[feature_col].unique():
            feature_data = patient_data[patient_data[feature_col] == feature]
            observed_count = len(feature_data)
            if observed_count < target_frequency:
                synthetic_entries += (target_frequency - observed_count)
        synthetic_data_proportion[patient_id] = synthetic_entries / total_entries
    return synthetic_data_proportion

In [ ]:
def calculate_patient_synthetic_data_proportion(patient_data, target_frequency=13):
    patient_synthetic_data_proportion = {}
    for patient_id, group in patient_data.groupby('subject_id'):
        observed_count = group['label'].count()
        total_expected = len(group) * target_frequency
        synthetic_count = total_expected - observed_count
        synthetic_proportion = synthetic_count / total_expected
        patient_synthetic_data_proportion[patient_id] = synthetic_proportion
    return patient_synthetic_data_proportion

In [ ]:
def compare_synthetic_data_proportions(train_df, test_df, patient_id_col, feature_col, target_frequency=13):
    train_synthetic_data_proportion = calculate_synthetic_data_proportion(train_df, patient_id_col, feature_col, target_frequency)
    test_synthetic_data_proportion = calculate_synthetic_data_proportion(test_df, patient_id_col, feature_col, target_frequency)

    train_avg_synthetic_data_proportion = sum(train_synthetic_data_proportion.values()) / len(train_synthetic_data_proportion)
    test_avg_synthetic_data_proportion = sum(test_synthetic_data_proportion.values()) / len(test_synthetic_data_proportion)

    print(f"Train Average Synthetic Data Proportion: {train_avg_synthetic_data_proportion:.4f}")
    print(f"Test Average Synthetic Data Proportion: {test_avg_synthetic_data_proportion:.4f}")

def compare_class_distributions(train_df, test_df, target_col):
    train_class_distribution = train_df[target_col].value_counts(normalize=True)
    test_class_distribution = test_df[target_col].value_counts(normalize=True)

    print("Train Class Distribution:")
    print(train_class_distribution)
    print("\nTest Class Distribution:")
    print(test_class_distribution)

def compare_feature_distributions(train_df, test_df, feature_cols):
    train_feature_stats = train_df[feature_cols].describe().transpose()
    test_feature_stats = test_df[feature_cols].describe().transpose()

    comparison_df = train_feature_stats[['mean', 'std']].merge(
        test_feature_stats[['mean', 'std']],
        left_index=True,
        right_index=True,
        suffixes=('_train', '_test')
    )

    print("\nFeature Statistics Comparison:")
    print(comparison_df)

In [ ]:
compare_synthetic_data_proportions(train_data, test_data, 'subject_id', 'feature_label')

Train Average Synthetic Data Proportion: 5.0855
Test Average Synthetic Data Proportion: 5.1079


*(Running on lower threshold)*

In [ ]:
compare_synthetic_data_proportions(train_data, test_data, 'subject_id', 'feature_label')

Train Average Synthetic Data Proportion: 8.5599
Test Average Synthetic Data Proportion: 8.6051


The amount of synthetic data is higher than previous as expected.

The proportion of synthetic data that would be required in the maximal case (i.e. no feature subsets) is similar, meaning the split is well stratified.

We will now compare class distributions

In [ ]:
def compare_class_distributions(train_df, test_df, target_col):
    train_class_distribution = train_df[target_col].value_counts(normalize=True)
    test_class_distribution = test_df[target_col].value_counts(normalize=True)

    print("Train Class Distribution:")
    print(train_class_distribution)
    print("\nTest Class Distribution:")
    print(test_class_distribution)

In [ ]:
compare_class_distributions(train_data, test_data, 'extubation_failure')

Train Class Distribution:
extubation_failure
0    0.668082
1    0.331918
Name: proportion, dtype: float64

Test Class Distribution:
extubation_failure
0    0.66622
1    0.33378
Name: proportion, dtype: float64


*After new threshold*

In [ ]:
compare_class_distributions(train_data, test_data, 'extubation_failure')

Train Class Distribution:
extubation_failure
0    0.670421
1    0.329579
Name: proportion, dtype: float64

Test Class Distribution:
extubation_failure
0    0.666536
1    0.333464
Name: proportion, dtype: float64


The class distribution is near identical between the two and reflects the inherent class distribution of the dataset which is ideal.

Finally, we will check the feature distributions between the train and test sets.

In [ ]:
def pivot_features(data):
    """Pivot the DataFrame so that each feature label in the 'label' column becomes a column."""
    pivoted_df = data.pivot_table(index='subject_id', columns='feature_label', values='valuenum', aggfunc='mean').reset_index()
    return pivoted_df

def compare_feature_distributions(train_df, test_df, feature_labels):
    # Pivot the train and test dataframes
    train_pivoted = pivot_features(train_df)
    test_pivoted = pivot_features(test_df)

    # Calculate statistics for the pivoted features
    train_feature_stats = train_pivoted[feature_labels].describe().transpose()
    test_feature_stats = test_pivoted[feature_labels].describe().transpose()

    # Merge the statistics from train and test dataframes
    comparison_df = train_feature_stats[['mean', 'std']].merge(
        test_feature_stats[['mean', 'std']],
        left_index=True,
        right_index=True,
        suffixes=('_train', '_test')
    )

    # Print the comparison dataframe in a readable format
    print("\nFeature Statistics Comparison:")
    print(comparison_df)

In [ ]:
feature_cols = average_sampling_frequency_df['Feature'].tolist()

# Remove low freq features from feature cols
feature_cols = [x for x in feature_cols if x not in low_observed_features]

compare_feature_distributions(train_data, test_data, feature_cols)


Feature Statistics Comparison:
                                   mean_train     std_train   mean_test  \
feature_label                                                             
Heart Rate                          84.674881     16.131241   84.940555   
Respiratory Rate                    19.094851      7.168183   18.855197   
O2 saturation pulseoxymetry         97.569262      2.406151   97.512783   
Arterial Blood Pressure mean        81.807956     15.146365   82.306634   
Arterial Blood Pressure diastolic   60.503410     18.254267   61.305475   
Arterial Blood Pressure systolic   123.695378     19.759156  123.007560   
Inspired O2 Fraction                42.971156      9.731068   42.922510   
GCS - Eye Opening                    3.191998      0.915553    3.152262   
GCS - Motor Response                 5.335014      1.246194    5.222507   
Tidal Volume (observed)            469.480313    146.227187  481.555807   
Minute Volume                        8.783212      6.849875    8.692

It should be noted that some of these features are not inherently numerical e.g. Ventilator Mode, RAS and GCS and hence their statistics should not be considered.

*After new threshold*

In [ ]:
feature_cols = average_sampling_frequency_df['Feature'].tolist()

# Remove low freq features from feature cols
feature_cols = [x for x in feature_cols if x not in low_observed_features]

compare_feature_distributions(train_data, test_data, feature_cols)


Feature Statistics Comparison:
                                   mean_train     std_train   mean_test  \
feature_label                                                             
Heart Rate                          84.674881     16.131241   84.940555   
Respiratory Rate                    19.094851      7.168183   18.855197   
O2 saturation pulseoxymetry         97.569262      2.406151   97.512783   
Arterial Blood Pressure mean        81.807956     15.146365   82.306634   
Arterial Blood Pressure diastolic   60.503410     18.254267   61.305475   
Arterial Blood Pressure systolic   123.695378     19.759156  123.007560   
Inspired O2 Fraction                42.971156      9.731068   42.922510   
GCS - Eye Opening                    3.191998      0.915553    3.152262   
GCS - Motor Response                 5.335014      1.246194    5.222507   
Tidal Volume (observed)            469.480313    146.227187  481.555807   
Minute Volume                        8.783212      6.849875    8.692

### **Step 4: Remove any features that clinically are not useful**

Having spoken to clinical practitioners, there are some features that are not useful to look at from this list.

These were identified to be:
- Ventilator Mode: as this is machine specific and therefore not generalisable to other machines

In [ ]:
# Remove any rows where the feature_label is ventilator mode
train_data = train_data[train_data['feature_label'] != 'Ventilator Mode']
test_data = test_data[test_data['feature_label'] != 'Ventilator Mode']

print(train_data['feature_label'].unique())
print(test_data['feature_label'].unique())

['Inspired O2 Fraction' 'Tidal Volume (observed)'
 'Tidal Volume (spontaneous)' 'Minute Volume' 'Peak Insp. Pressure'
 'Mean Airway Pressure' 'EtCO2' 'Heart Rate' 'Respiratory Rate'
 'GCS - Eye Opening' 'GCS - Motor Response' 'O2 saturation pulseoxymetry'
 'Richmond-RAS Scale' 'Arterial Blood Pressure systolic'
 'Arterial Blood Pressure diastolic' 'Arterial Blood Pressure mean'
 'Temperature Fahrenheit' 'Hematocrit (serum)' 'Sodium (serum)'
 'Potassium (serum)' 'Arterial O2 pressure' 'Arterial CO2 Pressure'
 'PH (Arterial)' 'Arterial Base Excess' 'Arterial O2 Saturation'
 'Ionized Calcium' 'Lactic Acid' 'Hemoglobin' 'WBC' 'Creatinine (serum)'
 'Glucose (serum)' 'Platelet Count' 'Plateau Pressure'
 'Cardiac Output (CCO)']
['O2 saturation pulseoxymetry' 'Inspired O2 Fraction'
 'Tidal Volume (observed)' 'Tidal Volume (spontaneous)' 'Minute Volume'
 'Peak Insp. Pressure' 'Mean Airway Pressure' 'Temperature Fahrenheit'
 'Richmond-RAS Scale' 'GCS - Eye Opening' 'GCS - Motor Response'
 'Heart

### **Step 5: Outlier Detection and Removal**

We can now remove outliers using the ranges provided by MIMIC or where not available the mean ± 3 std as the lower and upper bound.

For features that are inherently categorical but represented numerically we will not remove outliers as the values refer to a specific meaning.

In [ ]:
# Load the d_items table
items_path = '/content/drive/MyDrive/MSc_Final_Project/02_data_analysis/mimic/mimic-iv-2.2-raw-data/icu/d_items.csv'
items_df = pd.read_csv(items_path)
items_df.head()

,itemid,label,abbreviation,linksto,category,unitname,param_type,lownormalvalue,highnormalvalue
0,220001,Problem List,Problem List,chartevents,General,NaN,Text,NaN,NaN
1,220003,ICU Admission date,ICU Admission date,datetimeevents,ADT,NaN,Date and time,NaN,NaN
2,220045,Heart Rate,HR,chartevents,Routine Vital Signs,bpm,Numeric,NaN,NaN
3,220046,Heart rate Alarm - High,HR Alarm - High,chartevents,Alarms,bpm,Numeric,NaN,NaN
4,220047,Heart Rate Alarm - Low,HR Alarm - Low,chartevents,Alarms,bpm,Numeric,NaN,NaN


In [ ]:
# Rename label column to feature label
items_df = items_df.rename(columns={'label': 'feature_label'})
items_df.head()

,itemid,feature_label,abbreviation,linksto,category,unitname,param_type,lownormalvalue,highnormalvalue
0,220001,Problem List,Problem List,chartevents,General,NaN,Text,NaN,NaN
1,220003,ICU Admission date,ICU Admission date,datetimeevents,ADT,NaN,Date and time,NaN,NaN
2,220045,Heart Rate,HR,chartevents,Routine Vital Signs,bpm,Numeric,NaN,NaN
3,220046,Heart rate Alarm - High,HR Alarm - High,chartevents,Alarms,bpm,Numeric,NaN,NaN
4,220047,Heart Rate Alarm - Low,HR Alarm - Low,chartevents,Alarms,bpm,Numeric,NaN,NaN


In [ ]:
# For the unique labels in the patient data, extract the lownormalvalue and highnormalvalue
mimic_ranges = pd.merge(train_data, items_df, left_on='feature_label', right_on='feature_label', how='left')

# For each label, extract the lownormalvalue and highnormalvalue
mimic_ranges = mimic_ranges[['feature_label', 'lownormalvalue', 'highnormalvalue']].drop_duplicates()

# Remove any rows where both lownormalvalue and highnormalvalue are NaN
mimic_ranges

,feature_label,lownormalvalue,highnormalvalue
0,Inspired O2 Fraction,NaN,NaN
1,Tidal Volume (observed),299.0,750.0
2,Tidal Volume (spontaneous),299.0,750.0
3,Minute Volume,NaN,12.1
4,Peak Insp. Pressure,NaN,NaN
5,Mean Airway Pressure,NaN,NaN
6,EtCO2,NaN,NaN
7,Heart Rate,NaN,NaN
8,Respiratory Rate,NaN,NaN
9,GCS - Eye Opening,NaN,NaN


In [ ]:
# For each feature we can calculate statistics
train_feature_stats = train_data.groupby('feature_label').agg({
    'valuenum': ['min', 'max', 'mean', 'median', 'std']
}).reset_index()

train_feature_stats

feature_label valuenum                                 \
                                           min        max        mean  median   
0                Arterial Base Excess   -30.00  999999.00  511.350841    1.00   
1   Arterial Blood Pressure diastolic     0.00    6545.00   60.370794   58.00   
2        Arterial Blood Pressure mean   -19.00     353.00   81.459553   79.00   
3    Arterial Blood Pressure systolic     0.00     252.00  123.458819  121.00   
4               Arterial CO2 Pressure    16.00      89.00   41.048930   40.00   
5              Arterial O2 Saturation    84.00     100.00   96.452412   97.00   
6                Arterial O2 pressure    16.00     525.00  112.679409  106.00   
7                Cardiac Output (CCO)     1.60      12.60    5.340353    5.20   
8                  Creatinine (serum)     0.20      13.20    1.540634    1.10   
9                               EtCO2    17.00      62.00   37.704482   38.00   
10                  GCS - Eye Opening     1.00       4.00    3.148353    3.00   
11               GCS - Motor Response     1.00       6.00    5.366133    6.00   
12                    Glucose (serum)    23.00     435.00  137.264907  127.00   
13                         Heart Rate     0.00     182.00   84.905516   83.00   
14                 Hematocrit (serum)    15.20      49.20   28.730592   28.20   
15                         Hemoglobin     5.10      15.30    9.563626    9.40   
16               Inspired O2 Fraction     0.00     100.00   43.433712   40.00   
17                    Ionized Calcium     0.78       1.40    1.119098    1.12   
18                        Lactic Acid     0.50      19.20    1.911050    1.30   
19               Mean Airway Pressure     0.00      28.00    7.500339    7.00   
20                      Minute Volume     0.00     777.00    8.818543    8.30   
21        O2 saturation pulseoxymetry     0.00     100.00   97.572667   98.00   
22                      PH (Arterial)     6.80       7.58    7.412026    7.42   
23                Peak Insp. Pressure     0.00    2523.00   14.192709   12.00   
24                   Plateau Pressure     0.00      41.00   18.454348   18.00   
25                     Platelet Count     6.00     941.00  169.752613  140.00   
26                  Potassium (serum)     2.70       7.80    4.077043    4.00   
27                   Respiratory Rate     0.00    2325.00   19.175607   19.00   
28                 Richmond-RAS Scale    -5.00       4.00   -1.015962   -1.00   
29                     Sodium (serum)   117.00     164.00  140.113913  140.00   
30             Temperature Fahrenheit     0.00     109.50   98.887857   98.80   
31            Tidal Volume (observed)     0.00    1360.00  472.025328  452.00   
32         Tidal Volume (spontaneous)     0.00   52540.00  505.749019  446.00   
33                                WBC     0.10      67.10   12.176058   10.90   

                  
             std  
0   22581.883769  
1      56.363856  
2      18.811019  
3      24.151498  
4       8.622495  
5       2.021430  
6      38.317571  
7       1.429666  
8       1.290646  
9       7.134713  
10      0.971382  
11      1.211888  
12     48.484985  
13     18.082079  
14      4.656687  
15      1.640590  
16     11.918554  
17      0.078290  
18      2.332621  
19      3.200701  
20     10.296240  
21      3.012675  
22      0.074149  
23     33.633063  
24      5.170608  
25    123.392201  
26      0.573276  
27     15.833371  
28      1.599150  
29      5.655475  
30      2.100771  
31    163.021589  
32    910.968934  
33      6.980690

In [ ]:
train_feature_stats.columns = ['_'.join(col).strip() for col in train_feature_stats.columns.values]
train_feature_stats.rename(columns={'label_': 'label'}, inplace=True)
train_feature_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34 entries, 0 to 33
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   feature_label_   34 non-null     object 
 1   valuenum_min     34 non-null     float64
 2   valuenum_max     34 non-null     float64
 3   valuenum_mean    34 non-null     float64
 4   valuenum_median  34 non-null     float64
 5   valuenum_std     34 non-null     float64
dtypes: float64(5), object(1)
memory usage: 1.7+ KB


In [ ]:
merged_ranges = pd.merge(mimic_ranges, train_feature_stats, left_on='feature_label', right_on='feature_label_')
merged_ranges

,feature_label,lownormalvalue,highnormalvalue,feature_label_,valuenum_min,valuenum_max,valuenum_mean,valuenum_median,valuenum_std
0,Inspired O2 Fraction,NaN,NaN,Inspired O2 Fraction,0.00,100.00,43.433712,40.00,11.918554
1,Tidal Volume (observed),299.0,750.0,Tidal Volume (observed),0.00,1360.00,472.025328,452.00,163.021589
2,Tidal Volume (spontaneous),299.0,750.0,Tidal Volume (spontaneous),0.00,52540.00,505.749019,446.00,910.968934
3,Minute Volume,NaN,12.1,Minute Volume,0.00,777.00,8.818543,8.30,10.296240
4,Peak Insp. Pressure,NaN,NaN,Peak Insp. Pressure,0.00,2523.00,14.192709,12.00,33.633063
5,Mean Airway Pressure,NaN,NaN,Mean Airway Pressure,0.00,28.00,7.500339,7.00,3.200701
6,EtCO2,NaN,NaN,EtCO2,17.00,62.00,37.704482,38.00,7.134713
7,Heart Rate,NaN,NaN,Heart Rate,0.00,182.00,84.905516,83.00,18.082079
8,Respiratory Rate,NaN,NaN,Respiratory Rate,0.00,2325.00,19.175607,19.00,15.833371
9,GCS - Eye Opening,NaN,NaN,GCS - Eye Opening,1.00,4.00,3.148353,3.00,0.971382


Mean ± 3 std caluclation

In [ ]:
# Create reasonable ranges
k = 3

# Update lownormalvalue and highnormalvalue
merged_ranges['lownormalvalue'] = merged_ranges.apply(
    lambda row: row['lownormalvalue'] if not pd.isna(row['lownormalvalue'])
    else max(row['valuenum_min'], row['valuenum_mean'] - k * row['valuenum_std']),
    axis=1
)

merged_ranges['highnormalvalue'] = merged_ranges.apply(
    lambda row: row['highnormalvalue'] if not pd.isna(row['highnormalvalue'])
    else min(row['valuenum_max'], row['valuenum_mean'] + k * row['valuenum_std']),
    axis=1
)

In [ ]:
# Drop the extra columns
merged_train_ranges = merged_ranges[['feature_label','lownormalvalue', 'highnormalvalue']]

merged_train_ranges

,feature_label,lownormalvalue,highnormalvalue
0,Inspired O2 Fraction,7.678051,79.189374
1,Tidal Volume (observed),299.000000,750.000000
2,Tidal Volume (spontaneous),299.000000,750.000000
3,Minute Volume,0.000000,12.100000
4,Peak Insp. Pressure,0.000000,115.091899
5,Mean Airway Pressure,0.000000,17.102442
6,EtCO2,17.000000,59.108621
7,Heart Rate,30.659278,139.151754
8,Respiratory Rate,0.000000,66.675720
9,GCS - Eye Opening,1.000000,4.000000


We need to fix Arterial Base Excess so that it is within a reasonable range as there seems to be a large outlier skewing the std and mean.

Having discussed with Dr Mayur, we will use a range of -10 to +10 to bring the range into more reasonable values and remove any extreme outliers.

In [ ]:
# Adjust Arterial Base Excess	values to -5 and + 5
merged_train_ranges.loc[merged_train_ranges['feature_label'] == 'Arterial Base Excess', 'lownormalvalue'] = -10
merged_train_ranges

merged_train_ranges.loc[merged_train_ranges['feature_label'] == 'Arterial Base Excess', 'highnormalvalue'] = 10
merged_train_ranges

,feature_label,lownormalvalue,highnormalvalue
0,Inspired O2 Fraction,7.678051,79.189374
1,Tidal Volume (observed),299.000000,750.000000
2,Tidal Volume (spontaneous),299.000000,750.000000
3,Minute Volume,0.000000,12.100000
4,Peak Insp. Pressure,0.000000,115.091899
5,Mean Airway Pressure,0.000000,17.102442
6,EtCO2,17.000000,59.108621
7,Heart Rate,30.659278,139.151754
8,Respiratory Rate,0.000000,66.675720
9,GCS - Eye Opening,1.000000,4.000000


**Define categorical features**

From the features that we have left, the inherently categorical features are:
- GCS - Eye Opening
- GCS - Motor Response
- Richmond-RAS Scale

These will be excluded from outlier removal as the bounds are redundant.

Clinically validating these ranges with Doctor - these are reasonable and we can proceed in removing outliers outside this range and replacing them with NaNs to be later filled.

In [ ]:
categorical_features = ['GCS - Eye Opening', 'GCS - Motor Response', 'Richmond-RAS Scale']

In [ ]:
# Function to set outliers as NaN
def set_outliers_to_nan(df, mimic_df, categorical_features):
    for i, row in mimic_df.iterrows():
        if row['feature_label'] not in categorical_features:
            feature_mask = df['feature_label'] == row['feature_label']
            df.loc[feature_mask & ((df['valuenum'] < row['lownormalvalue']) | (df['valuenum'] > row['highnormalvalue'])), 'valuenum'] = np.nan
    return df

In [ ]:
train_data_copy = train_data.copy()
test_data_copy = test_data.copy()

In [ ]:
train_data_outliers_removed = set_outliers_to_nan(train_data_copy, merged_train_ranges, categorical_features)
test_data_outliers_removed = set_outliers_to_nan(test_data_copy, merged_train_ranges, categorical_features)

We can now visualise how many data points were outliers.

In [ ]:
# Function to calculate the percentage of NaN values per feature
def calculate_nan_stats(df, label_col='feature_label', value_col='valuenum'):
    stats = []
    for label in df[label_col].unique():
        feature_mask = df[label_col] == label
        total_points = feature_mask.sum()
        nan_points = df.loc[feature_mask, value_col].isna().sum()
        percentage_nan = (nan_points / total_points) * 100
        stats.append({
            'label': label,
            'total_data_points': total_points,
            'nan_data_points': nan_points,
            'percentage_nan': percentage_nan
        })
    return pd.DataFrame(stats)

In [ ]:
train_nan_stats = calculate_nan_stats(train_data_outliers_removed)
test_nan_stats = calculate_nan_stats(test_data_outliers_removed)

print("Train NaN Stats:")
print(train_nan_stats)
print("\nTest NaN Stats:")
print(test_nan_stats)

Train NaN Stats:
                                label  total_data_points  nan_data_points  \
0                Inspired O2 Fraction               7920              214   
1             Tidal Volume (observed)               6079              843   
2          Tidal Volume (spontaneous)               5154              815   
3                       Minute Volume               6019              657   
4                 Peak Insp. Pressure               5747                1   
5                Mean Airway Pressure               5906               78   
6                               EtCO2                714                2   
7                          Heart Rate              24856              160   
8                    Respiratory Rate              24737                5   
9                   GCS - Eye Opening               6134                0   
10               GCS - Motor Response               6118                0   
11        O2 saturation pulseoxymetry              24805   

It seems arterial blood pressure values have a lot of outliers. This may be due to the bounds provided by MIMIC being too conservative but we did not want to overrule them.

Cardica output has a very high number of outliers but these are based on the bounds given by MIMIC.

In [ ]:
# Calculate the percentage of data points removed in train set
total_data_points = len(train_data_copy)
nan_data_points = train_data_outliers_removed['valuenum'].isna().sum()
percentage_nan_overall = (nan_data_points / total_data_points) * 100

print(f"Percentage of NaN data in train set: {percentage_nan_overall:.2f}%")

Percentage of NaN data in train set: 7.94%


In [ ]:
# Calculate the percentage of data points removed in test set
total_data_points = len(test_data_copy)
nan_data_points = test_data_outliers_removed['valuenum'].isna().sum()
percentage_nan_overall = (nan_data_points / total_data_points) * 100

print(f"Percentage of NaN data in test set: {percentage_nan_overall:.2f}%")

Percentage of NaN data in test set: 8.03%


Give reasons as to why Arterial Blood pressure has the most outliers. Physiological variablility, conservative bounds etc.

In [ ]:
# Save train and test data
train_data_outliers_removed.to_parquet('/content/drive/MyDrive/MSc_Final_Project/02_data_analysis/mimic/data_analysis/datasets/07_data_preprocessing/03_feature_set_3/pre_processing/03_train_data_f3_outliers_removed_v2.parquet')
test_data_outliers_removed.to_parquet('/content/drive/MyDrive/MSc_Final_Project/02_data_analysis/mimic/data_analysis/datasets/07_data_preprocessing/03_feature_set_3/pre_processing/03_test_data_f3_outliers_removed_v2.parquet')